In [1]:
%pylab inline
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
def read_lines(day, test=False):
    if test:
        with open(f'data/2021/{day}_test.txt', 'r') as f:
            lines = f.readlines()
    else:
        with open(f'data/2021/{day}.txt', 'r') as f:
            lines = f.readlines()
    return [line.strip() for line in lines]

def lines_as_ints(lines):
    return list(map(int, lines))

def debug(*args, **kwargs):
    if DEBUG:
        print(*args)
        
def bool_to_int(array):
    return int(''.join(['1' if x else '0' for x in array]), 2)

In [3]:
def day1(lines):
    vals = lines_as_ints(lines)
    part1 = sum(np.diff(vals) > 0)
    
    windowed = np.convolve(vals, np.ones(3), mode='valid')
    part2 = sum(diff(windowed) > 0)
    return (part1, part2)

In [4]:
def day2(lines):
    pairs = [line.split() for line in lines]
    pairs = [(d, int(v)) for (d,v) in pairs]
    
    x = 0; z = 0
    for (d, v) in pairs:
        if d == 'forward':
            x = x + v
        elif d == 'down':
            z = z + v
        elif d == 'up':
            z = z - v
    p1 = x * z
    debug(x, z, x * z)
            
    x = 0; z = 0; aim = 0
    for (d, v) in pairs:
        if d == 'forward':
            x = x + v
            z = z + aim * v
        elif d == 'down':
            aim = aim + v
        elif d == 'up':
            aim = aim - v
    p2 = x * z
    debug(x, z, x * z)
    
    return (p1, p2)

In [5]:
def day3(lines):
    vals = np.array(list(map(lambda line: [int(x) for x in line], lines)), dtype=np.bool)
    means = (np.mean(vals, axis=0) > 0.5)
    p1 = bool_to_int(means) * bool_to_int(~means)

    vals2 = vals
    for ii in range(0, vals2.shape[1]):
        crit = (np.mean(vals2[:,ii]) >= 0.5)
        vals2 = vals2[vals2[:,ii] == crit,:]
        if vals2.shape[0] == 1:
            break
    o2 = bool_to_int(vals2[0])

    vals2 = vals
    for ii in range(0, vals2.shape[1]):
        crit = (np.mean(vals2[:,ii]) < 0.5)
        vals2 = vals2[vals2[:,ii] == crit,:]
        if vals2.shape[0] == 1:
            break
    co2 = bool_to_int(vals2[0])
    p2 = o2 * co2

    return (p1, p2)

In [6]:
def day4(lines):
    draw = list(map(int, lines[0].split(',')))
    grids = lines[2:]
    all_grids = []
    cur_grid = []
    for line in grids:
        if line != '':
            vals = line.strip().split(' ')
            vals = [int(x) for x in vals if x != '']
            cur_grid.append(vals)
        else:
            all_grids.append(cur_grid)
            cur_grid = []
    all_grids.append(cur_grid)
    grids = np.array(all_grids)
    
    g = grids.copy()

    found = False
    board = None
    row = None
    col = None

    for ii in range(0, len(draw)):
        g[g == draw[ii]] = -1

        if np.any(np.all(g == -1, axis=1)):
            found = True
            board = np.where(np.all(g == -1, axis=1))[0][0]
            col = np.where(np.all(g == -1, axis=1))[0][0]
            break

        if np.any(np.all(g == -1, axis=2)):
            found = True
            board = np.where(np.all(g == -1, axis=2))[0][0]
            row = np.where(np.all(g == -1, axis=2))[1][0]
            break
    board_sum = np.sum(g[board, g[board] != -1])
    p1 = board_sum * draw[ii]
    
    g = grids.copy()

    solved_order = []
    solved = set()

    for ii in range(0, len(draw)):
        g[g == draw[ii]] = -1

        if np.any(np.all(g == -1, axis=1)):
            boards = np.where(np.all(g == -1, axis=1))[0]
            for b in boards:
                if b not in solved:
                    solved_order.append(b)
                    solved.add(b)

        if np.any(np.all(g == -1, axis=2)):
            boards = np.where(np.all(g == -1, axis=2))[0]
            for b in boards:
                if b not in solved:
                    solved_order.append(b)
                    solved.add(b)

        if len(solved) == g.shape[0]:
            break
    p2 = np.sum(g[solved_order[-1], g[solved_order[-1]] != -1]) * draw[ii]
    return (p1, p2)

In [7]:
from collections import Counter


def day5(lines):
    split = [line.split('->') for line in lines]

    c = Counter()

    for line in lines:
        s = line.split('->')
        start = s[0].split(',')
        end = s[1].split(',')
        start = (int(start[0]), int(start[1]))
        end = (int(end[0]), int(end[1]))

        if start[0] == end[0]:
            ss = min(start[1], end[1])
            ee = max(start[1], end[1])
            for ii in range(ss, ee+1):
                c.update([(start[0], ii)])
        if start[1] == end[1]:
            ss = min(start[0], end[0])
            ee = max(start[0], end[0])
            for ii in range(ss, ee+1):
                c.update([(ii, start[1])])
    p1 = 0
    for (k, v) in c.items():
        if v > 1:
            p1 = p1 + 1
    
    c = Counter()
    for line in lines:
        s = line.split('->')
        start = s[0].split(',')
        end = s[1].split(',')
        start = (int(start[0]), int(start[1]))
        end = (int(end[0]), int(end[1]))

        if start[0] == end[0]:
            ss = min(start[1], end[1])
            ee = max(start[1], end[1])
            for ii in range(ss, ee+1):
                c.update([(start[0], ii)])
        elif start[1] == end[1]:
            ss = min(start[0], end[0])
            ee = max(start[0], end[0])
            for ii in range(ss, ee+1):
                c.update([(ii, start[1])])
        else:
            if end[0] < start[0]:
                (start, end) = (end, start)
            m = (end[1] - start[1]) / (end[0] - start[0])
            xx = start[0]; yy = start[1]
            while xx != end[0] + 1:
                c.update([(xx,yy)])
                xx = xx + 1
                yy = yy + int(m)
    p2 = 0
    for (k, v) in c.items():
        if v > 1:
            p2 = p2 + 1            
    return (p1, p2)

In [8]:
DEBUG = False

run = [
    (day1, 1521, 1543),
    (day2, 1451208, 1620141160),
    (day3, 3885894, 4375225),
    (day4, 8580, 9576), 
    (day5, 6113, 20373)
]
    

for vv in run:
    idx = int(vv[0].__name__[3:])
    
    (p1, p2) = vv[0](read_lines(idx))
    
    print(vv[0].__name__)
    print('part 1: ', p1)
    print('part 2: ', p2)

day1
part 1:  1521
part 2:  1543
day2
part 1:  1451208
part 2:  1620141160
day3
part 1:  3885894
part 2:  4375225
day4
part 1:  8580
part 2:  9576
day5
part 1:  6113
part 2:  20373
